In [1]:
import os
import tensorflow as tf
import dask
import dask.array as da
import numpy as np

In [2]:
train_x = da.from_npy_stack('/home/skyolia/JupyterProjects/classification/cifar_10/data/train_x')
train_y = da.from_npy_stack('/home/skyolia/JupyterProjects/classification/cifar_10/data/train_y')
test_x = da.from_npy_stack('/home/skyolia/JupyterProjects/classification/cifar_10/data/test_x')
test_y = da.from_npy_stack('/home/skyolia/JupyterProjects/classification/cifar_10/data/test_y')
train_x, train_y, test_x, test_y

(dask.array<from-npy-stack, shape=(50000, 32, 32, 3), dtype=float64, chunksize=(10000, 32, 32, 3)>,
 dask.array<from-npy-stack, shape=(50000, 1), dtype=float64, chunksize=(10000, 1)>,
 dask.array<from-npy-stack, shape=(10000, 32, 32, 3), dtype=float64, chunksize=(1000, 32, 32, 3)>,
 dask.array<from-npy-stack, shape=(10000, 1), dtype=float64, chunksize=(1000, 1)>)

In [3]:
class CIFAR10Sequence(tf.keras.utils.Sequence):

    def __init__(self, x, y, batch_size):
        self.x, self.y = x, y
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y
        
class LRTensorBoard(tf.keras.callbacks.TensorBoard):
    '''
    Add learning rate evolution to Tensorboard
    '''
    def __init__(self, log_dir):
        super().__init__(log_dir=log_dir)

    def on_epoch_end(self, epoch, logs=None):
        logs['lr'] = tf.keras.backend.eval(self.model.optimizer.lr)
        super().on_epoch_end(epoch, logs)

def build_block(input_layer, filters, norm=True, k=[3,3]):
    layer = tf.keras.layers.Conv2D(filters, kernel_size=(k[0], k[1]), padding='same', use_bias=not norm, kernel_initializer='glorot_uniform')(input_layer)
    if norm:
        layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.Activation('elu')(layer)
    return layer

def build_model(num_class):
    image_input = tf.keras.Input(shape=(32, 32, 3), name='input_layer')
    conv_1 = build_block(image_input, 48)
    conv_2 = build_block(conv_1, 48)
    pool_1 = tf.keras.layers.MaxPooling2D(padding='same')(conv_2)
    drop_1 = tf.keras.layers.Dropout(0.6)(pool_1)
    conv_3 = build_block(drop_1, 96)
    conv_4 = build_block(conv_3, 96)
    pool_2 = tf.keras.layers.MaxPooling2D(padding='same')(conv_4)
    drop_2 = tf.keras.layers.Dropout(0.6)(pool_2)
    conv_5 = build_block(drop_2, 192)
    conv_6 = build_block(conv_5, 192)
    pool_3 = tf.keras.layers.MaxPooling2D(padding='same')(conv_6)
    drop_3 = tf.keras.layers.Dropout(0.6)(pool_3)
    conv_7 = build_block(drop_3, 192, False, [1,1])
    drop_4 = tf.keras.layers.Dropout(0.25)(conv_7)
    gap = tf.keras.layers.GlobalAvgPool2D()(drop_4)
    logits = tf.keras.layers.Dense(units=num_class, activation='softmax', bias_initializer='ones', kernel_initializer='glorot_uniform')(gap)
    model = tf.keras.Model(inputs=image_input, outputs=logits)
    return model

In [4]:
model = build_model(10)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 48)        1296      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 48)        192       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 48)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 48)        20736     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 48)        192       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 48)        0         
__________

In [5]:
model.load_weights("day_1.weights.best.hdf5")
sgd = tf.keras.optimizers.SGD(lr=1.5)
model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
scores = model.evaluate(test_x, test_y)
scores

10000/10000 [==============================] - 4s 446us/step


[0.4764824456214905, 0.8424]

In [5]:
epochs, lr, batch_size = 10000, 1., 256
steps_per_epoch = int(np.ceil(train_y.shape[0]/batch_size))
filepath="day_1.weights.best.hdf5"

train_generator = CIFAR10Sequence(train_x, train_y, batch_size)
test_generator = CIFAR10Sequence(test_x, test_y, batch_size)
tblr = LRTensorBoard(log_dir=os.getcwd())
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=0.008, mode='min', cooldown=1, verbose=1)

sgd = tf.keras.optimizers.SGD(lr=lr) # 
model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#model.load_weights(filepath)
model.fit_generator(generator=train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=int(np.ceil(test_y.shape[0]/batch_size)),
                    use_multiprocessing=True,
                    workers=12,
                    shuffle=True,
                    #initial_epoch=65,
                    callbacks=[checkpoint, reduce_lr, tblr])


Epoch 1/10000
195/196 [============================>.] - ETA: 0s - loss: 7.5661 - acc: 0.1585
Epoch 00001: val_acc improved from -inf to 0.24880, saving model to day_1.weights.best.hdf5
196/196 [==============================] - 37s 190ms/step - loss: 7.5382 - acc: 0.1591 - val_loss: 2.0150 - val_acc: 0.2488
Epoch 2/10000
195/196 [============================>.] - ETA: 0s - loss: 1.9065 - acc: 0.2777
Epoch 00002: val_acc improved from 0.24880 to 0.28880, saving model to day_1.weights.best.hdf5
196/196 [==============================] - 33s 167ms/step - loss: 1.9057 - acc: 0.2780 - val_loss: 1.8430 - val_acc: 0.2888
Epoch 3/10000
195/196 [============================>.] - ETA: 0s - loss: 1.7301 - acc: 0.3492
Epoch 00003: val_acc improved from 0.28880 to 0.36810, saving model to day_1.weights.best.hdf5
196/196 [==============================] - 33s 169ms/step - loss: 1.7297 - acc: 0.3495 - val_loss: 1.6792 - val_acc: 0.3681
Epoch 4/10000
195/196 [============================>.] - ETA: 0s

Process ForkPoolWorker-22229:
Process ForkPoolWorker-22233:
Process ForkPoolWorker-22227:
Process ForkPoolWorker-22232:
Process ForkPoolWorker-22231:
Process ForkPoolWorker-22238:
Process ForkPoolWorker-22244:
Traceback (most recent call last):
Process ForkPoolWorker-22230:
Traceback (most recent call last):
Process ForkPoolWorker-22228:
Process ForkPoolWorker-22240:
Process ForkPoolWorker-22225:
Process ForkPoolWorker-22226:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/skyol

KeyboardInterrupt: 